In [ ]:
!pip install pandas-ods-reader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ezodf: filename=ezodf-0.3.2-py2.py3-none-any.whl size=49002 sha256=c64c364a925cc69483b251cdb690280694ea1cd0801ae8b7f5bf9fc1417dcaf9
  Stored in directory: /root/.cache/pip/wheels/50/09/62/ea2c44e6ffdd067d6c8d3f557af0734a195bd252d6901f23ca
Successfully built ezodf


In [ ]:
import pandas as pd
import statsmodels.api as sm
import csv
import matplotlib.pyplot as plt
import os
from pandas_ods_reader import read_ods
import numpy as np
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, LassoCV, LassoLarsCV, Lasso

import xgboost as xgb

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.chdir("/content/gdrive/MyDrive/ColabNotebooks/PredictScore")

In [ ]:
data_path = "/content/gdrive/MyDrive/ColabNotebooks/PredictScore/Data"

# Modify data

---



In [ ]:
df = pd.read_csv(data_path + "/annonimized.csv")
df.columns = [col.replace("concat('it001',`","").replace("`)","").strip() for col in df.columns]
df.columns = [col.replace("concat('it001',","").replace(")","").strip() for col in df.columns]
df['created_at'] = df['created_at'].apply(lambda x: x.split(' ')[1])
df['updated_at'] = df['updated_at'].apply(lambda x: x.split(' ')[1])
df['time_diff'] = df['updated_at'].astype('timedelta64[s]') - df['created_at'].astype('timedelta64[s]')
df = df.drop(['created_at', 'updated_at'], axis=1)

tbtl_df = read_ods(data_path + "/tbtl-public.ods")
tbtl_df['TBTL'] = tbtl_df['TBTL'].astype(float)

In [ ]:
import json
df['judgement'] = df['judgement'].apply(json.loads)
df[['times', 'mems', 'verdicts','remain_judgement']] = df['judgement'].apply(pd.Series)
df['remain_judgement'] = df['remain_judgement'].fillna('[]')
df['times'] = df['times'].fillna('[]')
df['mems'] = df['mems'].fillna('[]')
df['verdicts'] = df['verdicts'].fillna('[]')
df = df.drop(['judgement'], axis=1)

In [ ]:
df.head()

,assignment_id,problem_id,username,is_final,status,pre_score,coefficient,language_id,time_diff,times,mems,verdicts,remain_judgement
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,0 days 00:04:54,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",{'WRONG': 10},[]
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,0,SCORE,0,100,it0012,0 days 00:00:10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",{'WRONG': 10},[]
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,0 days 00:00:09,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[],[]
3,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,0 days 00:00:09,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[],[]
4,90ce27571176d87961b565d5ef4b3de33ede04ac,7a6e5ca470ff47c3b5048f240c4738de71010c78,ed9eaeb6a707f50154024b24d7efcb874a9795dd,1,SCORE,10000,100,it0012,0 days 00:00:10,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[],[]


In [ ]:
tbtl_df

,username,TBTL
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,7.24
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,8.11
2,01122b3ef7e59b84189e65985305f575d6bdf83c,7.30
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,8.63
4,013de369c439ab0ead8aa7da64423aa395a8be39,8.20
...,...,...
794,840d07858c03f80f4695056e2cc7d0c474b83a25,7.67
795,844f5db2e7e31ae51eba025480679ed7e4708ac6,7.54
796,845acd04a77b3d1b623f255d9f9f8eae90892dab,7.49
797,8460eaaf887a6289fb156f7562fb739ba8e9629e,8.94


In [ ]:
th_df = pd.read_csv(data_path + "/th-public.csv")
qt_df = pd.read_csv(data_path + "/qt-public.csv")
ck_df = pd.read_csv(data_path + "/ck-public.csv")
th_df.columns = ['username', 'TH']
qt_df.columns = ['username', 'QT']
ck_df.columns = ['username', 'CK']
th_df['TH'] = pd.to_numeric(th_df['TH'], errors='coerce')
qt_df['QT'] = pd.to_numeric(qt_df['QT'], errors='coerce')

In [ ]:
th_df

,username,TH
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.0
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,8.5
2,01122b3ef7e59b84189e65985305f575d6bdf83c,7.0
3,013de369c439ab0ead8aa7da64423aa395a8be39,10.0
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,6.0
...,...,...
744,840d07858c03f80f4695056e2cc7d0c474b83a25,6.0
745,844f5db2e7e31ae51eba025480679ed7e4708ac6,8.5
746,845acd04a77b3d1b623f255d9f9f8eae90892dab,5.5
747,8460eaaf887a6289fb156f7562fb739ba8e9629e,10.0


In [ ]:
qt_df

,username,QT
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,7.5
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,7.0
2,01122b3ef7e59b84189e65985305f575d6bdf83c,9.0
3,013de369c439ab0ead8aa7da64423aa395a8be39,10.0
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,9.0
...,...,...
749,840d07858c03f80f4695056e2cc7d0c474b83a25,9.0
750,844f5db2e7e31ae51eba025480679ed7e4708ac6,8.0
751,845acd04a77b3d1b623f255d9f9f8eae90892dab,6.5
752,8460eaaf887a6289fb156f7562fb739ba8e9629e,10.0


In [ ]:
ck_df

,username,CK
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.5
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,6.5
2,01122b3ef7e59b84189e65985305f575d6bdf83c,4.5
3,013de369c439ab0ead8aa7da64423aa395a8be39,5.0
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,4.0
...,...,...
746,840d07858c03f80f4695056e2cc7d0c474b83a25,5.5
747,844f5db2e7e31ae51eba025480679ed7e4708ac6,6.0
748,845acd04a77b3d1b623f255d9f9f8eae90892dab,6.0
749,8460eaaf887a6289fb156f7562fb739ba8e9629e,7.0


In [ ]:
df = pd.merge(df, tbtl_df, on='username')
df = pd.merge(df, th_df, on='username')
df = pd.merge(df, qt_df, on='username')
df = pd.merge(df, ck_df, on='username')

In [ ]:
df.isna().sum()

,0
assignment_id,0
problem_id,0
username,0
is_final,0
status,0
pre_score,0
coefficient,0
language_id,0
time_diff,0
times,0


In [ ]:
df = df.dropna()

In [ ]:
# df['TBTL'] = df['TBTL'].fillna(df['TBTL'].mean())
# df['TH'] = df['TH'].fillna(df['TH'].mean())
# df['QT'] = df['QT'].fillna(df['QT'].mean())
# df['CK'] = df['CK'].fillna(df['CK'].mean())

In [ ]:
df.head()
#Khong dung assignment_id, username, is_final, coefficient, language_id, created_at, updated_at
#1 assignment gom nhieu problem_id
#username khong anh huong so diem cua mot sinh vien
#is_final chi cho biet do la ket qua sinh vien chon cuoi cung, khong anh huong den qua trinh, nang suat lam bai

,assignment_id,problem_id,username,is_final,status,pre_score,coefficient,language_id,time_diff,times,mems,verdicts,remain_judgement,TH
0,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,0bd2037bf68a97753e5e67ab55dac026a649f279,0,Compilation Error,0,100,it0012,0 days 00:01:04,[],[],{' forbidden phrase: &quot;include&quot; is b...,[],4.5
1,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,0bd2037bf68a97753e5e67ab55dac026a649f279,0,Compilation Error,0,100,it0012,0 days 00:00:01,[],[],{' forbidden phrase: &quot;include&quot; is b...,[],4.5
2,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,0bd2037bf68a97753e5e67ab55dac026a649f279,0,Compilation Error,0,100,it0012,0 days 00:19:31,[],[],{' forbidden phrase: &quot;include&quot; is b...,[],4.5
3,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,0bd2037bf68a97753e5e67ab55dac026a649f279,0,Compilation Error,0,100,it0012,0 days 00:00:01,[],[],{' forbidden phrase: &quot;include&quot; is b...,[],4.5
4,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,0bd2037bf68a97753e5e67ab55dac026a649f279,0,Compilation Error,0,100,it0012,0 days 00:00:01,[],[],{' forbidden phrase: &quot;include&quot; is b...,[],4.5


In [ ]:
#Save





df.to_csv('CK.csv', index=False)

In [ ]:
df.dtypes.unique().tolist()

[dtype('O'), dtype('int64'), dtype('<m8[s]'), dtype('float64')]

In [ ]:
th_df.dtypes.unique().tolist()
# qt_df.dtypes.unique().tolist()
# ck_df.dtypes.unique().tolist()

[dtype('O'), dtype('float64')]

In [ ]:
cat_cols = df.dtypes.index[df.dtypes == 'O'].tolist()
num_cols = df.dtypes.index[df.dtypes != 'O'].tolist()

In [ ]:
cat_cols

['assignment_id',
 'problem_id',
 'username',
 'status',
 'language_id',
 'times',
 'mems',
 'verdicts',
 'remain_judgement']

In [ ]:
num_cols

['is_final', 'pre_score', 'coefficient', 'time_diff', 'TBTL']

In [ ]:
df[num_cols].describe()

,is_final,pre_score,coefficient,time_diff,TBTL
count,156842.000000,156842.000000,156842.000000,156842,156842.000000
mean,0.319226,5059.272178,99.643527,0 days 00:03:46,7.954942
std,0.466178,4379.925855,5.432805,0 days 01:41:00,0.828311
min,0.000000,0.000000,0.000000,-1 days +00:01:13,2.520000
25%,0.000000,0.000000,100.000000,0 days 00:00:07,7.470000
50%,0.000000,5714.000000,100.000000,0 days 00:00:09,8.020000
75%,1.000000,10000.000000,100.000000,0 days 00:00:22,8.590000
max,1.000000,10000.000000,100.000000,0 days 23:05:59,9.610000


In [ ]:
df[cat_cols].describe()

KeyboardInterrupt: 

In [ ]:
print(df.isnull().sum())

assignment_id    0
problem_id       0
username         0
is_final         0
status           0
pre_score        0
coefficient      0
language_id      0
created_at       0
updated_at       0
judgement        0
TBTL             0
dtype: int64


# Model

---



In [ ]:
df = pd.read_csv("CK.csv")

In [ ]:
# df['time_diff'] = pd.to_timedelta(df['time_diff'])

In [ ]:
df = df[df['is_final'] != 0]
df.head()

,assignment_id,problem_id,username,is_final,status,pre_score,coefficient,language_id,time_diff,times,mems,verdicts,remain_judgement,CK
11,90ce27571176d87961b565d5ef4b3de33ede04ac,789454427dd4097a14749e3dde63346b7a8d3811,0bd2037bf68a97753e5e67ab55dac026a649f279,1,SCORE,10000,100,it0012,0 days 00:00:07,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[],[],1.0
14,90ce27571176d87961b565d5ef4b3de33ede04ac,bf96fbdc5f499538c3e2bfbec5779c8a14b0a9ff,0bd2037bf68a97753e5e67ab55dac026a649f279,1,SCORE,10000,100,it0012,0 days 00:00:08,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[],[],1.0
21,90ce27571176d87961b565d5ef4b3de33ede04ac,9ec58869b69a632e3cba27924e342ba40eb378b0,0bd2037bf68a97753e5e67ab55dac026a649f279,1,SCORE,10000,100,it0012,0 days 00:00:07,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[],[],1.0
29,90ce27571176d87961b565d5ef4b3de33ede04ac,1d824e02c40fecdae25d89c0357410997dfbf799,0bd2037bf68a97753e5e67ab55dac026a649f279,1,SCORE,10000,100,it0012,0 days 00:00:07,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[],[],1.0
33,90ce27571176d87961b565d5ef4b3de33ede04ac,f86c7e74f00cdc9777cba2ed6041a482869c8040,0bd2037bf68a97753e5e67ab55dac026a649f279,1,SCORE,10000,100,it0012,0 days 00:00:07,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[],[],1.0


In [ ]:
cat_cols = df.dtypes.index[df.dtypes == 'O'].tolist()

In [ ]:
cat_cols

['verdicts']

In [ ]:
#X = df.drop(['assignment_id','username','is_final','language_id', 'remain_judgement', 'TBTL'], axis=1)
X = df.drop(['is_final','status', 'language_id','remain_judgement','times','mems','time_diff','coefficient','CK'], axis=1)
y = df['CK']

In [ ]:
X.isna().sum()

,0
assignment_id,0
username,0
pre_score,0
coefficient,0
time_diff,0
times,0
mems,0
verdicts,0


In [208]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
# Assuming 'X_train' contains the features with non-numeric data
label_encoder = LabelEncoder()
#X_train['username'] = label_encoder.fit_transform(X_train['username'])
#X['assignment_id'] = label_encoder.fit_transform(X['assignment_id'])
X['username'] = label_encoder.fit_transform(X['username'])
X['problem_id'] = label_encoder.fit_transform(X['problem_id'])
#X['status'] = label_encoder.fit_transform(X['status'])
#X['times'] = label_encoder.fit_transform(X['times'])
#X['mems'] = label_encoder.fit_transform(X['mems'])
X['verdicts'] = label_encoder.fit_transform(X['verdicts'])
# X['time_diff'] =  X['time_diff'].dt.seconds
#X['time_diff'] = label_encoder.fit_transform(X['time_diff'])
# X_train['created_at'] = label_encoder.fit_transform(X_train['created_at'])
# X_train['updated_at'] = label_encoder.fit_transform(X_train['updated_at'])
# X_train['judgement'] = label_encoder.fit_transform(X_train['judgement'])

#X_test['username'] = label_encoder.fit_transform(X_test['username'])
# X_test['assignment_id'] = label_encoder.fit_transform(X_test['assignment_id'])
# X_test['problem_id'] = label_encoder.fit_transform(X_test['problem_id'])
# X_test['status'] = label_encoder.fit_transform(X_test['status'])
# X_test['times'] = label_encoder.fit_transform(X_test['times'])
# X_test['mems'] = label_encoder.fit_transform(X_test['mems'])
# X_test['verdicts'] = label_encoder.fit_transform(X_test['verdicts'])
# X_test['created_at'] = label_encoder.fit_transform(X_test['created_at'])
# X_test['updated_at'] = label_encoder.fit_transform(X_test['updated_at'])
# X_test['judgement'] = label_encoder.fit_transform(X_test['judgement'])

In [ ]:
print(X,y)

        assignment_id  problem_id  username  pre_score  verdicts
11                114         187        72      10000         0
14                114         306        72      10000         0
21                114         258        72      10000         0
29                114          47        72      10000         0
33                114         414        72      10000         0
...               ...         ...       ...        ...       ...
155929             75         151       201      10000         0
155930             75          86       201      10000         0
155932             75         343       201      10000         0
155933             75         220       201      10000         0
155934             75         365       201      10000         0

[49815 rows x 5 columns] 11        1.0
14        1.0
21        1.0
29        1.0
33        1.0
         ... 
155929    6.0
155930    6.0
155932    6.0
155933    6.0
155934    6.0
Name: CK, Length: 49815, dtype: float64


In [209]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train.isna().sum()

,0
assignment_id,0
problem_id,0
status,0
pre_score,0
coefficient,0
time_diff,0
times,0
mems,0
verdicts,0
TH,0


In [ ]:
X_test.isna().sum()

,0
assignment_id,0
problem_id,0
status,0
pre_score,0
coefficient,0
time_diff,0
times,0
mems,0
verdicts,0
TH,0


In [215]:
model = GradientBoostingRegressor(max_depth=7)
#model = SVR()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)
#problem_id is index_column: R-squared: 0.008728912282578993
#has index_column: R-squared: 0.008910622325077111
#has is_final: R-squared: 0.008784783981205413
#have TH, CK, QT: R-squared: 0.01046209779002738
#has assignment_id: R-squared: 0.009942128602842826

Mean Squared Error: 1.0589284155103464
R-squared: 0.7530393559447197


In [ ]:
def epsilon_insensitive_loss(actual, predicted, epsilon):
    return np.maximum(0, np.abs(actual - predicted) - epsilon)

In [ ]:
epsilon_insensitive_loss(y_test, y_pred, 0.1)

,TBTL
44470,0.775659
16421,0.527689
127075,1.587770
55400,1.658454
103436,1.094864
...,...
54873,1.148370
49482,0.007879
142708,0.000000
157366,0.253624


In [ ]:
data = pd.read_csv(data_path + "/annonimized.csv")
data.columns = [col.replace("concat('it001',`","").replace("`)","").strip() for col in data.columns]
data.columns = [col.replace("concat('it001',","").replace(")","").strip() for col in data.columns]
data_predicted = data

X_to_predict = data_predicted.drop(['language_id', 'username'], axis=1).copy()  # Make a copy to avoid SettingWithCopyWarning
#X_to_predict_temp = X_to_predict['username']
#X_to_predict['username'] = label_encoder.fit_transform(X_to_predict['username'])
X_to_predict['assignment_id'] = label_encoder.fit_transform(X_to_predict['assignment_id'])
X_to_predict['problem_id'] = label_encoder.fit_transform(X_to_predict['problem_id'])
X_to_predict['status'] = label_encoder.fit_transform(X_to_predict['status'])
X_to_predict['created_at'] = label_encoder.fit_transform(X_to_predict['created_at'])
X_to_predict['updated_at'] = label_encoder.fit_transform(X_to_predict['updated_at'])
X_to_predict['judgement'] = label_encoder.fit_transform(X_to_predict['judgement'])
# Dự đoán điểm cho các username trong X
predictions = model.predict(X_to_predict)
# Hiển thị dự đoán theo username
predictions_df = pd.DataFrame({'username': data['username'], 'predicted_score': predictions})
print(predictions_df)

In [ ]:
average_predicted_scores = predictions_df.groupby('username')['predicted_score'].mean()

print(average_predicted_scores)

In [ ]:
data = pd.read_csv(data_path + "/annonimized.csv")
data.columns = [col.replace("concat('it001',`","").replace("`)","").strip() for col in data.columns]
data.columns = [col.replace("concat('it001',","").replace(")","").strip() for col in data.columns]

data['created_at'] = data['created_at'].apply(lambda x: x.split(' ')[1])
data['updated_at'] = data['updated_at'].apply(lambda x: x.split(' ')[1])
data['time_diff'] = data['updated_at'].astype('timedelta64[s]') - data['created_at'].astype('timedelta64[s]')
data = data.drop(['created_at', 'updated_at'], axis=1)

import json
data['judgement'] = data['judgement'].apply(json.loads)
data[['times', 'mems', 'verdicts','remain_judgement']] = data['judgement'].apply(pd.Series)
data['remain_judgement'] = data['remain_judgement'].fillna('[]')
data['times'] = data['times'].fillna('[]')
data['mems'] = data['mems'].fillna('[]')
data['verdicts'] = data['verdicts'].fillna('[]')
data = data.drop(['judgement'], axis=1)


data_predicted = data

tbtl_df = read_ods(data_path + "/tbtl-public.ods")
tbtl_df['TBTL'] = tbtl_df['TBTL'].astype(float)

tbtl_df = read_ods(data_path + "/tbtl-public.ods")
tbtl_df['TBTL'] = tbtl_df['TBTL'].astype(float)

th_df = pd.read_csv(data_path + "/th-public.csv")
qt_df = pd.read_csv(data_path + "/qt-public.csv")
ck_df = pd.read_csv(data_path + "/ck-public.csv")
th_df.columns = ['username', 'TH']
qt_df.columns = ['username', 'QT']
ck_df.columns = ['username', 'CK']
th_df['TH'] = pd.to_numeric(th_df['TH'], errors='coerce')
qt_df['QT'] = pd.to_numeric(qt_df['QT'], errors='coerce')

In [216]:
X_to_predict = data.copy()

In [ ]:
data['coefficient'].unique()

array([100,   0,  85,  75])

In [217]:
#X_to_predict = pd.merge(X_to_predict, tbtl_df, how='left', on='username')
#X_to_predict = pd.merge(X_to_predict, th_df, how='left', on='username')
#X_to_predict = pd.merge(X_to_predict, qt_df, how='left', on='username')
X_to_predict = pd.merge(X_to_predict, ck_df, how='left', on='username')

In [218]:
#X_to_predict['TBTL'] = X_to_predict['TBTL'].fillna(tbtl_df['TBTL'].mean().round(2))
#X_to_predict['TH'] = X_to_predict['TH'].fillna(th_df['TH'].mean().round(2))
#X_to_predict['QT'] = X_to_predict['QT'].fillna(qt_df['QT'].mean().round(2))
X_to_predict['CK'] = X_to_predict['CK'].fillna(ck_df['CK'].mean().round(2))

In [219]:
X_temp = X_to_predict.copy()
#X_to_predict = X_to_predict.drop(['assignment_id', 'username', 'is_final', 'coefficient', 'language_id', 'remain_judgement', 'TBTL'], axis=1)
X_to_predict = X_to_predict.drop(['assignment_id','is_final','status', 'language_id','remain_judgement','times','mems','time_diff','coefficient','CK'], axis=1)

In [222]:
for col in ['times', 'mems', 'verdicts']:
  X_to_predict[col] = X_to_predict[col].astype(str)

In [ ]:
X_to_predict.head()

,problem_id,status,pre_score,coefficient,time_diff,times,mems,verdicts,TH,QT,CK
0,789454427dd4097a14749e3dde63346b7a8d3811,Compilation Error,0,100,0 days 00:01:04,[],[],{'\n\nforbidden phrase: &quot;include&quot; is...,4.5,6.0,1.0
1,789454427dd4097a14749e3dde63346b7a8d3811,Compilation Error,0,100,0 days 00:00:01,[],[],{'\n\nforbidden phrase: &quot;include&quot; is...,4.5,6.0,1.0
2,789454427dd4097a14749e3dde63346b7a8d3811,Compilation Error,0,100,0 days 00:19:31,[],[],{'\n\nforbidden phrase: &quot;include&quot; is...,4.5,6.0,1.0
3,789454427dd4097a14749e3dde63346b7a8d3811,Compilation Error,0,100,0 days 00:00:01,[],[],{'\n\nforbidden phrase: &quot;include&quot; is...,4.5,6.0,1.0
4,789454427dd4097a14749e3dde63346b7a8d3811,Compilation Error,0,100,0 days 00:00:01,[],[],{'\n\nforbidden phrase: &quot;include&quot; is...,4.5,6.0,1.0


In [223]:
cat_cols = X_to_predict.dtypes.index[X_to_predict.dtypes == 'O'].tolist()

In [ ]:
cat_cols

['problem_id', 'username', 'times', 'mems', 'verdicts']

In [224]:
label_encoder = LabelEncoder()
#X_to_predict['assignment_id'] = label_encoder.fit_transform(X_to_predict['assignment_id'])
X_to_predict['username'] = label_encoder.fit_transform(X_to_predict['username'])
X_to_predict['problem_id'] = label_encoder.fit_transform(X_to_predict['problem_id'])
#X_to_predict['status'] = label_encoder.fit_transform(X_to_predict['status'])
#X_to_predict['times'] = label_encoder.fit_transform(X_to_predict['times'])
#X_to_predict['mems'] = label_encoder.fit_transform(X_to_predict['mems'])
X_to_predict['verdicts'] = label_encoder.fit_transform(X_to_predict['verdicts'])
#X_to_predict['time_diff'] = label_encoder.fit_transform(X_to_predict['time_diff'])

In [225]:
predictions = model.predict(X_to_predict)
# Hiển thị dự đoán theo username
predictions_df = pd.DataFrame({'username': X_temp['username'], 'predicted_score': predictions})
df_grouped = predictions_df.groupby('username')['predicted_score'].mean().reset_index()

In [226]:
df_grouped['predicted_score'] = df_grouped['predicted_score'].round(2)

In [227]:
df_grouped

,username,predicted_score
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.83
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,6.29
2,01122b3ef7e59b84189e65985305f575d6bdf83c,4.74
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,4.98
4,013de369c439ab0ead8aa7da64423aa395a8be39,4.50
...,...,...
1484,feb8a2859a011c59efd22ed419cb69288fe03627,3.22
1485,fef4a3263ed9a8ab14d457694bb8fd86ccd98312,3.11
1486,ff12d6e2ab80696ed8e22fbe5497e96c68d29076,2.92
1487,ff3fa2ec64294f37ae968159f810ebeda7966c51,2.90


In [ ]:
df_grouped.to_csv('predict_tbtl.csv', index=False, header=False)